In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop
from keras import optimizers
import sys
from keras.callbacks import LambdaCallback
import random
import matplotlib.pyplot as plt


import numpy as np
import os
import time

Using TensorFlow backend.


In [2]:
tweet_data=pd.read_csv('../Load_Tweets/data/tweet_data.csv')

In [3]:
tweet_text = tweet_data['TEXT']

In [4]:
tweet_text_all = tweet_data['TEXT'].str.cat(sep=' ')

In [5]:
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didn\'t she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio!West Palm Beach, FL at noon:[URL]Cincinnati, OH this 7:30pm:[URL] Wow, @CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders. Hillary &amp; CNN FRAUD! Thank you Texas! If you haven\'t registered to VOTE- today is your last day. Go to: [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY. You can register now at: [URL] and get out to… [URL] DON\'T LET HER FOOL US AGAIN. [URL] Crooked\'s State Dept gave special attention to "Friends of Bill" after the Haiti Earthquake. Unbelieva

In [6]:
print ('Length of text: {} characters'.format(len(tweet_text_all)))

Length of text: 1513769 characters


In [7]:
# Take a look at the first 250 characters in text
print(tweet_text_all[:250])

PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been t


In [8]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [9]:
import re
tweet_text_all=re.sub('(\w)\.(\w)', '\\1. \\2', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4})(\w)', ' \\1 \\2 \\3', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4}) ', '\\1 \\2 ',tweet_text_all) #fdj... fdj ...
tweet_text_all=re.sub(' (\.{2,10})(\w)', ' \\1 \\2',tweet_text_all) #...fdj ... fdj
tweet_text_all=re.sub('[)(]', '', tweet_text_all) #can make paranteses their own token

In [10]:
tweet_text_all=re.sub('!(.)',' ! \\1', tweet_text_all)
tweet_text_all=re.sub('\"', '', tweet_text_all) # can keep the qoutes
tweet_text_all=re.sub("\'", '', tweet_text_all)
tweet_text_all=re.sub("\”", '', tweet_text_all)
tweet_text_all=re.sub("\‘", '', tweet_text_all)
tweet_text_all=re.sub("\’", '', tweet_text_all)
tweet_text_all=re.sub("#(.)", '# \\1', tweet_text_all) 
tweet_text_all=re.sub("@(.)", '@ \\1', tweet_text_all)
tweet_text_all=re.sub("(.),", '\\1 ,', tweet_text_all)
tweet_text_all=re.sub(",(.)", ', \\1', tweet_text_all)
tweet_text_all=re.sub("(.):", '\\1 :', tweet_text_all)


In [11]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [12]:
# tweet_text_all=re.sub('([^.])\.([^.])', '\\1 EOS\\2', tweet_text_all) 
# tweet_text_all = tweet_text_all.replace(".", ' <EOS>')
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest !  Rigged system !  Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West Palm Beach ,  FL at noon :[URL]Cincinnati ,  OH this 7 :30pm :[URL] Wow ,  @ CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders. Hillary &amp; CNN FRAUD !  Thank you Texas !  If you havent registered to VOTE- today is your last day. Go to : [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY. You can register now at : [URL] and get out to… [URL] DONT LET HER FOOL US AGAIN. [URL] Crookeds State Dept gave special attention to Friends of Bill after the Haiti Earthqua

In [13]:
list_of_words=pd.Series(tweet_text_all.split())
print ('Length of text: {} words'.format(len(list_of_words)))

Length of text: 283502 words


In [14]:
#number of unique words
word_vocab=sorted(set(list_of_words))
print ('{} unique words'.format(len(word_vocab)))

23452 unique words


In [15]:
# The unique characters in the file
vocab = sorted(set(tweet_text_all))
print ('{} unique characters'.format(len(vocab)))

362 unique characters


In [16]:
# #replace apostrophes in dictionary keys 
# for i in range (len(list_of_words)):
#     list_of_words[i]=list_of_words[i].replace("‘", '').replace("’", '').replace("'", '').replace('"', '')

In [17]:
# #see how many words after apostrophe deletion
# word_vocab=sorted(set(list_of_words))
# print ('{} unique words'.format(len(word_vocab)))

In [18]:
word_vocab.append('UNK')

In [19]:
word_vocab[-1]

'UNK'

In [20]:
#create forward and reverse word index
word2idx = dict((c, i) for i, c in enumerate(word_vocab))
# idx2word = dict((i, c) for i, c in enumerate(word_vocab,1 ))
idx2word= np.asarray(word_vocab)

text_as_int = np.array([word2idx[w] for w in list_of_words])

In [21]:
print('{')
for word,_ in zip(word2idx, range(2000)):
    print('  {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print('  ...\n}')

{
  'Wont': 11244,
  'siege?': 20563,
  'COS': 2350,
  'Superman': 9784,
  'Trump-like': 10356,
  'Missouri?': 6891,
  'Strengthening': 9725,
  'TOUGH': 9925,
  'nominates': 18053,
  'Law-Abiding': 6129,
  'billion': 12387,
  'stands': 20927,
  'premise': 19007,
  'believers': 12313,
  'Stories': 9702,
  'foxandfriends.': 15466,
  'Country?': 2971,
  'conciliatory.': 13322,
  're-election.': 19490,
  'handled.': 15893,
  'frontrunner': 15517,
  'tear': 21375,
  'forgivenes': 15412,
  'Razak': 8458,
  'disgrac': 14206,
  'Tr…': 10437,
  'many': 17481,
  '“chief': 23184,
  'project.': 19191,
  'requirements': 19856,
  'Daines': 3238,
  'WaysandMeansGOP': 11093,
  'DESPERATELY': 3144,
  'Louvre': 6331,
  'WAR': 10904,
  'Cubs': 3057,
  'Workforce': 11257,
  'GovWalker.': 4696,
  'MeetTheTrumps': 6752,
  'cont': 13433,
  'Covington': 2988,
  'Egg': 3788,
  '49': 669,
  '“A': 22891,
  'now…': 18118,
  'LOVE': 6057,
  'Rates': 8444,
  'biker': 12383,
  'gentleman': 15634,
  'Refugees': 8531,

In [22]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- words mapped to int ---- > {}'.format(repr(' '.join(list_of_words[:13])), text_as_int[:13]))

'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest' ---- words mapped to int ---- > [ 7646  9912  7672  7684     3  3025 11376 10783 17240 18735 12696 21484
 14218]


In [23]:
idx2word

array(['!', '!#', '!Full', ..., '🦃', '\U0010fc00', 'UNK'], dtype='<U123')

In [24]:
# The maximum length sentence we want for a single input in characters
seq_length = 20
examples_per_epoch = len(list_of_words)//seq_length

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(5):
  print(idx2word[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.
PAY
TO
PLAY
POLITICS.
#


In [25]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2word[item.numpy()])))  


'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So'
'dishonest ! Rigged system ! Crooked Hillary Clinton likes to talk about the things she will do but she has been'
'there for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen before'
'and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West Palm'
'Beach , FL at noon :[URL]Cincinnati , OH this 7 :30pm :[URL] Wow , @ CNN Town Hall questions were given'


In [26]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [27]:
for input_example, target_example in  dataset.take(10):
  print ('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks.'
Target data: 'TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So'
Input data:  'dishonest ! Rigged system ! Crooked Hillary Clinton likes to talk about the things she will do but she has'
Target data: '! Rigged system ! Crooked Hillary Clinton likes to talk about the things she will do but she has been'
Input data:  'there for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen'
Target data: 'for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen before'
Input data:  'and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West'
Target data: 'will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West 

In [28]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2word[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2word[target_idx])))

Step    0
  input: 275 ('13')
  expected output: 20955 ('states')
Step    1
  input: 20955 ('states')
  expected output: 15957 ('have')
Step    2
  input: 15957 ('have')
  expected output: 22309 ('voter')
Step    3
  input: 22309 ('voter')
  expected output: 19690 ('registration')
Step    4
  input: 19690 ('registration')
  expected output: 13800 ('deadlines')


In [29]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 20), (64, 20)), types: (tf.int64, tf.int64)>

In [30]:
embeddings_index = {}
f = open('../word_embeding/glove.twitter.27B.25d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [31]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)+1

# The embedding dimension 
embedding_dim = 25

# Number of RNN units
rnn_units = 1024
embedding_matrix = np.zeros((vocab_size, embedding_dim)) 

for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [32]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  print ("on the GPU BABY!")
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

on the GPU BABY!


In [62]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):

    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                batch_input_shape=[batch_size, None,],
                                trainable=False),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
      ])
    return model

In [63]:
model = build_model(
  vocab_size = vocab_size, 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 25)            586350    
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (64, None, 1024)          3228672   
_________________________________________________________________
dense_3 (Dense)              (64, None, 23454)         24040350  
Total params: 27,855,372
Trainable params: 27,269,022
Non-trainable params: 586,350
_________________________________________________________________


In [65]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 20, 23454) # (batch_size, sequence_length, vocab_size)


In [66]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [67]:
sampled_indices

array([ 3338, 21062, 12570, 20658,   838,  8655, 12083, 12035, 11806,
        5799, 21378, 19625,  8730, 21511,  6491, 20478, 18436, 20802,
       12796,  9133])

In [68]:
print("Input: \n", repr(" ".join(idx2word[[input_example_batch[0]]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

Input: 
 'probably have a good relationship with Putin. We are doing MUCH better than any other country ! The Fake News'

Next Char Predictions: 
 'Declarations➡️[URL]➡️[URL]➡️[URL] string... breach. sleazeball 7pm. Rescue attending assembled and..... KEITH tech recovery[URL] Ridiculous theoretical MS-13 sheltered. overplaying southerner. careful SavannahGuthrie'


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [69]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 20, 23454)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.062791


In [70]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [71]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)


In [72]:
EPOCHS=60

In [73]:
history = model.fit(
    dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/60
221/221 [==============================] - 36s 163ms/step - loss: 7.8938
Epoch 2/60
221/221 [==============================] - 32s 146ms/step - loss: 7.6317
Epoch 3/60
221/221 [==============================] - 34s 154ms/step - loss: 7.5089
Epoch 4/60
221/221 [==============================] - 34s 155ms/step - loss: 7.3516
Epoch 5/60
221/221 [==============================] - 34s 154ms/step - loss: 7.1135
Epoch 6/60
221/221 [==============================] - 34s 154ms/step - loss: 6.7151
Epoch 7/60
221/221 [==============================] - 34s 155ms/step - loss: 6.5267
Epoch 8/60
221/221 [==============================] - 34s 155ms/step - loss: 6.3270
Epoch 9/60
221/221 [==============================] - 34s 155ms/step - loss: 6.1118
Epoch 10/60
221/221 [==============================] - 34s 155ms/step - loss: 5.8619
Epoch 11/60
221/221 [==============================] - 34s 154ms/step - loss: 5.5820
Epoch 12/60
221/221 [==============================] - 34s 155ms/step - lo

In [74]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_60'

In [75]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [76]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 25)             586350    
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (1, None, 1024)           3228672   
_________________________________________________________________
dense_4 (Dense)              (1, None, 23454)          24040350  
Total params: 27,855,372
Trainable params: 27,269,022
Non-trainable params: 586,350
_________________________________________________________________


In [77]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of words to generate
  num_generate = 30

  # Converting our start string to numbers (vectorizing) 
  if start_string!='' or ' ':
      for word in start_string.split():
        if word in word2idx:
          input_eval = [word2idx[word]]
          input_eval = tf.expand_dims(input_eval, 0)
        else:
            input_eval= [word2idx['UNK']]
            input_eval = tf.expand_dims(input_eval, 0)
  else: 
    input_eval= [word2idx['UNK']]
    input_eval = tf.expand_dims(input_eval, 0)

    

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2word[predicted_id])

  return (start_string + ' '.join(text_generated))

In [80]:
print(generate_text(model, start_string=u"when I"))

when IMASSIVE and I can win will Never a very difficult that in order off has become advantage of the U. S. New S. short is stronger El Paso all ,


In [81]:
model.save('../Saved_models/embedding_2_1.h5')